In [ ]:
import pandas as pd
import yaml

# Define the file path and folder names
file_path = r'F:\Guvi\Projects\Stock_Market_Analysis\data'
file_folder_name = [
    "2023-11", "2023-12", "2024-01", "2024-02", "2024-03", "2024-04", 
    "2024-05", "2024-06", "2024-07", "2024-08", "2024-09", "2024-10", 
    "2024-11"
]

#File selection based on the date range
File_start_date = "2023-11-01"
File_end_date = "2024-10-31"

#Date range for whole year
dates = pd.date_range(start=File_start_date, end=File_end_date, freq='D')

#To store the folder path
path_container = []

for folder in file_folder_name:
    year, month = folder.split("-")
    filtered_dates = dates[(dates.year == int(year)) & (dates.month == int(month))]
    for date in filtered_dates:
        path = f"{file_path}\\{folder}\\{date.strftime('%Y-%m-%d')}_05-30-00.yaml"
        path_container.append(path)

data_container = []

for yaml_file in path_container:
    try:
        with open(yaml_file,'r') as file:
            data = list(yaml.safe_load_all(file))
            data_container.extend(data)
    except FileNotFoundError:
        print(f"file not found {yaml_file}")
    except yaml.YAMLError as e:
        print(f"Error prasing YAML file {yaml_file}: {e}")

In [4]:
#Converting the CSV files into the DataFrame

Master_df = pd.DataFrame(data_container)

data_list = []

for row in range(Master_df.shape[0]):
    for column in range(Master_df.shape[1]):
        values=Master_df.iloc[row,column]
        if isinstance(values,dict):
            Master_data_conv=pd.DataFrame([values])
            data_list.append(Master_data_conv)

Master_data_list=pd.concat(data_list, ignore_index=True)

Master_data_list.to_csv("File1.csv", index=False)

In [ ]:
# Splitting the CSV file based on the Ticker symbol

Master_df_csv = pd.read_csv("File1.csv")

split_column = "Ticker"

csv_folder = r"F:\Guvi\Projects\Stock_Market_Analysis\CSV_files\\"

unique_values= Master_df_csv[split_column].unique()

for value in unique_values:
    splitted_df= Master_df_csv[Master_df_csv[split_column]==value]
    file_name=f"{csv_folder}{value}.csv"
    splitted_df.to_csv(file_name, index=False)
    print(f"Exported {file_name}")

In [6]:
# Concatinating all the CSV files into single CSV file

import pandas as pd

import os

#Folder that contains the CSV files
folder_path = r"F:\Guvi\Projects\Stock_Market_Analysis\CSV_files" 

#Getting all the csv files' path from the folder
csv_files = [cs for cs in os.listdir(folder_path) if cs.endswith('.csv')]

#Read and combaining all the csv
Master_df_list = []

for file in csv_files:
    file_path = os.path.join(folder_path, file)
    df = pd.read_csv(file_path)
    Master_df_list.append(df)

#Concatinating all the csv
final_df = pd.concat(Master_df_list, ignore_index=True)

#Saving all the CSV as one
final_df.to_csv(os.path.join(r"F:\Guvi\Projects\Stock_Market_Analysis\\", 'combined.csv'), index=False)

print("All CSV files combined successfully")

All CSV files combined successfully


In [9]:
import pandas as pd

Master_data = pd.read_csv('combined.csv')

Master_data.head()

,Ticker,close,date,high,low,month,open,volume
0,ADANIENT,2217.30,2023-11-01 05:30:00,2312.25,2207.00,2023-11,2299.00,1666579
1,ADANIENT,2215.30,2023-11-02 05:30:00,2297.95,2204.20,2023-11,2247.00,2340997
2,ADANIENT,2229.85,2023-11-03 05:30:00,2279.00,2215.00,2023-11,2215.30,1853300
3,ADANIENT,2246.00,2023-11-06 05:30:00,2260.00,2233.00,2023-11,2253.95,770061
4,ADANIENT,2233.35,2023-11-07 05:30:00,2260.00,2226.65,2023-11,2257.00,641554


In [10]:
# Handling Sector data and renameing the column for merging with master dataframe

df2=pd.read_csv('Sector_data.csv')

Sector = pd.DataFrame(df2)

Sector['Symbol'] = Sector['Symbol'].str.split(':').str[1].str.strip() # Cleaning the 'Symbol' column to match the 'Ticker' format in master dataframe

Sector.rename(columns={"COMPANY": "Company", "Symbol" : "Ticker", "sector" : "Sector"}, inplace=True) #Renaming the column 'Symbol' to 'Ticker' for consistency

Sector.head()

,Company,Sector,Ticker
0,ADANI ENTERPRISES,MISCELLANEOUS,ADANIGREEN
1,ADANI PORTS & SEZ,MISCELLANEOUS,ADANIPORTS
2,APOLLO HOSPITALS,MISCELLANEOUS,APOLLOHOSP
3,ASIAN PAINTS,PAINTS,ASIANPAINT
4,AXIS BANK,BANKING,AXISBANK


In [11]:
#Changing Ticker to TATACONSUMER in the Sector dataframe

Sector.loc[Sector['Ticker'] == 'TATACONSUMER', 'Ticker'] = 'TATACONSUM'

Sector.loc[Sector['Ticker'] == 'AIRTEL', 'Ticker'] = 'BHARTIARTL'

Sector.loc[Sector['Ticker'] == 'ADANIGREEN', 'Ticker'] = 'ADANIENT'

In [12]:
# Adding the ticker BRITANNIA to the Sector dataframe to match with the master dataframe
Sector.loc[len(Sector)] = ['BRITANNIA', 'FMCG', 'BRITANNIA']

In [13]:
Master_data=Master_data.merge(Sector[['Company', 'Sector', 'Ticker']], on='Ticker', how='left')

Master_data[['Ticker', 'Sector']].drop_duplicates()

Master_data[Master_data['Sector'].isna()][['Ticker', 'Sector']].drop_duplicates()

Master_data[['Ticker', 'Sector']].drop_duplicates().head()

Master_data.insert(1, 'Company', Master_data.pop('Company'))

Master_data.to_csv('Master_data.csv', index=False)

In [14]:
#Calculating daily volatility and standard deviation of each stock

Master_data['date'] = Master_data['date'].replace(" 05:30:00", "", regex=True)

Master_data['date']=pd.to_datetime(Master_data['date']) # Convert 'date' column to datetime format

Master_data['month'] = pd.to_datetime(Master_data['month'])

Master_data['month'] = Master_data['month'].dt.to_period('M').dt.to_timestamp()# Convert 'month' column to Period type

Master_data['Daily_Returns'] = ((Master_data['close'] - Master_data['close'].shift(1)))/Master_data['close'].shift(1) # Daily volatility formula

Master_data['Daily_Returns'] = Master_data['Daily_Returns'].fillna(0)

all_stock_volatility = Master_data.groupby('Ticker')['Daily_Returns'].std().round(2)*100 # Calculating standard deviation of volatility for each stock

Master_data['Std_Dev'] = Master_data['Ticker'].map(all_stock_volatility) # Mapping the standard deviation values back to the original dataframe

Ticker_and_std = Master_data[['Company','Ticker', 'Std_Dev']].drop_duplicates()

Ticker_and_std.to_csv('Ticker_and_std.csv', index=False) # Saving the ticker and its standard deviation to a CSV file


In [15]:
#Calculating cumulative return of each stock

Open_stock_price= Master_data.groupby('Ticker')['open'].first().reset_index() # Getting the opening price of each stock

close_stock_price= Master_data.groupby('Ticker')['close'].last().reset_index() # Getting the closing price of each stock

cumulative_return = pd.DataFrame({
    'Ticker': Open_stock_price['Ticker'],
    'Cumulative_amount': close_stock_price['close'] - Open_stock_price['open']
}) # Calculating cumulative return

cumulative_return.reset_index().to_csv('cumulative_return.csv', index=False) # Saving cumulative return to a CSV file

cumulative_return.head()

,Ticker,Cumulative_amount
0,ADANIENT,648.25
1,ADANIPORTS,591.20
2,APOLLOHOSP,2187.70
3,ASIANPAINT,-62.30
4,AXISBANK,181.70


In [16]:
#Mapping cumulative return to the Master_data dataframe

Master_data['Cum_Returns'] = Master_data['Ticker'].map(cumulative_return.set_index('Ticker')['Cumulative_amount'])

In [17]:
# Calculating the yearly average return

yearly_opening = Master_data.loc[Master_data.groupby('Company')['date'].idxmin(), 
                                 ['Company', 'date', 'open','Sector']].reset_index(drop=True)
yearly_Closing = Master_data.loc[Master_data.groupby('Company')['date'].idxmax(), 
                                 ['Company', 'date','close','Sector']].reset_index(drop=True)

yearly_returns = (yearly_Closing['close'] - yearly_opening['open']) / yearly_opening['open'] * 100

In [18]:
#Mapping the Sector wise return

Average_Yearly_return = yearly_opening.copy()
Average_Yearly_return['Closing_Date'] = yearly_Closing['date']
Average_Yearly_return['Close'] = yearly_Closing['close']
Average_Yearly_return['Yearly_Return_Percentage'] = yearly_returns.round(2)
Average_Yearly_return = Average_Yearly_return.sort_values('Company').reset_index(drop=True)

Average_Yearly_return = Average_Yearly_return.groupby('Sector')['Yearly_Return_Percentage'].mean().round(2).reset_index()

Average_Yearly_return.to_csv('Avg_Yearly_Return.csv')

In [19]:
Average_Yearly_return.head(5)

,Sector,Yearly_Return_Percentage
0,ALUMINIUM,49.63
1,AUTOMOBILES,53.29
2,BANKING,16.11
3,CEMENT,31.11
4,DEFENCE,113.01


In [20]:
Master_data["Avg_Yearly_return"] = Master_data['Sector'].map(dict(zip(Average_Yearly_return['Sector'], Average_Yearly_return['Yearly_Return_Percentage']))).round(2)

In [21]:
Corr_df = pd.DataFrame(Master_data, columns=['Company', 'close','date'])

# Pivot the data — set 'Company' as columns and 'close' as values
df_pivot = Corr_df.pivot_table(values='close', columns='Company', aggfunc=list)

# If you want each value on a separate row instead of a list:
df_pivot = Corr_df.set_index(Corr_df.groupby('Company').cumcount()).pivot(columns='Company', values='close')

df_pivot.reset_index(drop=True, inplace=True)

df_pivot.to_csv("Correlation_Closing_Stock.csv")


In [22]:
# Calculating monthly returns for each stock

monthly_avg = Master_data[['Ticker', 'date', 'close']].copy()

monthly_avg['date'] = pd.to_datetime(monthly_avg['date'])

monthly_avg['month'] = monthly_avg['date'].dt.to_period('M')

monthly_avg = monthly_avg.sort_values(['Ticker', 'date'])

monthly_opening = monthly_avg.groupby(['Ticker', 'month']).first().reset_index()

monthly_closing = monthly_avg.groupby(['Ticker', 'month']).last().reset_index()

monthly_data = pd.merge(monthly_opening, monthly_closing, on=['Ticker', 'month'], suffixes=('_open', '_close'))

monthly_data['monthly_return'] = ((monthly_data['close_close'] - monthly_data['close_open']) / monthly_data['close_open']) * 100
monthly_data['monthly_return'] = monthly_data['monthly_return'].round(2)

monthly_data['month'] = monthly_data['month'].dt.strftime('%b-%Y')

monthly_data = monthly_data.rename(columns={
    'close_open': 'open_price',
    'close_close': 'close_price',
    'date_open': 'open_date',
    'date_close': 'close_date'
})

monthly_data.to_csv('Monthly_return.csv', index=False)

In [23]:
Master_data['month'] = Master_data['month'] = (pd.to_datetime(Master_data['month']).dt.to_period('M')
                                               .dt.strftime('%b-%Y'))


In [24]:
Master_data = Master_data.merge(monthly_data[['Ticker', 'month', 'monthly_return']], 
                                on=['Ticker', 'month'],how='left')


In [25]:
Master_data.to_csv('Master_data.csv', index=False)

In [6]:
import pandas as pd

Master_data = pd.read_csv('Master_data.csv')

In [1]:
import pymysql

mydb = pymysql.connect(
 host="localhost",
 user="root",
 password=""
 
)

mycursor = mydb.cursor()

In [3]:
mycursor.execute("CREATE DATABASE IF NOT EXISTS stock_market_analysis")

1

In [2]:
mycursor.execute("USE Stock_Market_Analysis")

0

In [7]:
mycursor.execute("USE Stock_Market_Analysis")

mycursor.execute("""create table if not exists Master_data (
                        Company VARCHAR(255),
                        Ticker VARCHAR(50),
                        Sector VARCHAR(100),
                        date DATE,
                        month DATE,
                        open FLOAT,
                        high FLOAT,
                        low FLOAT,
                        close FLOAT,
                        volume BIGINT,
                        Daily_Returns FLOAT,
                        Std_Dev FLOAT,
                        Cum_Returns FLOAT,
                        Avg_Yearly_return FLOAT,
                        monthly_return FLOAT
                        )""")
for _,row in Master_data.iterrows():
    sql = """INSERT INTO Master_data (Company, Ticker, Sector, date, month, open, high, low, close, volume, 
            Daily_Returns, Std_Dev, Cum_Returns, Avg_Yearly_return, monthly_return) 
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"""
    val = (row['Company'], row['Ticker'], row['Sector'], row['date'], row['month'], row['open'], row['high'], 
           row['low'], row['close'], row['volume'], row['Daily_Returns'], row['Std_Dev'], row['Cum_Returns'], 
           row['Avg_Yearly_return'], row['monthly_return'])
    mycursor.execute(sql, val)
    mydb.commit()

In [26]:
mycursor.execute("""Create table if not exists Stock_Std_Dev (
                 COMPANY VARCHAR(255), 
                 Ticker VARCHAR(20), 
                 Std_Dev FLOAT
                 )""")

for _,row in Ticker_and_std.iterrows():
    sql = "INSERT INTO Stock_Std_Dev (Company, Ticker, Std_Dev) VALUES (%s, %s, %s)"
    val = (row['Company'], row['Ticker'], row['Std_Dev'])
    mycursor.execute(sql, val)
    mydb.commit()

In [27]:
mycursor.execute("""Create table if not exists Cumulative_Return
                 (Ticker VARCHAR(20), 
                 Cumulative_amount FLOAT
                 )""")
for _,row in cumulative_return.iterrows():
    sql = "INSERT INTO Cumulative_Return (Ticker, Cumulative_amount) VALUES (%s, %s)"
    val = (row['Ticker'], row['Cumulative_amount'])
    mycursor.execute(sql, val)
    mydb.commit()

In [28]:
mycursor.execute("""Create table if not exists Avg_Yearly_return
                 (Sector VARCHAR(100), 
                 Yearly_Return_Percentage FLOAT
                 )""")
for _,row in Average_Yearly_return.iterrows():
    sql = "INSERT INTO Avg_Yearly_return (Sector, Yearly_Return_Percentage) VALUES (%s, %s)"
    val = (row['Sector'], row['Yearly_Return_Percentage'])
    mycursor.execute(sql, val)
    mydb.commit()

In [29]:
#Correlation between the stock prices of different companies

columns = Master_data['Company'].unique().tolist()

mycursor.execute("CREATE TABLE IF NOT EXISTS Corr_Closing_Stocks "
"(" + ", ".join(f"`{col}` FLOAT" for col in columns)+ ")")

sql = f"""
INSERT INTO Corr_Closing_Stocks ({', '.join(f'`{c}`' for c in columns)})
VALUES ({', '.join(['%s'] * len(columns))})
"""

# Iterate and insert each row
for _, row in df_pivot.iterrows():
    val = tuple(row[col] for col in columns)
    mycursor.execute(sql, val)

mydb.commit()

In [32]:
#monthly breakdowns of the top-performing and worst-performing stocks

mycursor.execute("""create table if not exists Monthly_Stock_Returns
                 (Ticker VARCHAR(20),
                 month VARCHAR(15),
                 open_price FLOAT,
                 close_price FLOAT,
                 open_date DATE,
                 close_date DATE,
                 monthly_return_percentage FLOAT
                 )""")
for _,row in monthly_data.iterrows():
    sql = """INSERT INTO Monthly_Stock_Returns (Ticker, month, open_price, close_price, open_date, close_date, 
            monthly_return_percentage) VALUES (%s, %s, %s, %s, %s, %s, %s)"""
    val = (row['Ticker'], row['month'], row['open_price'], row['close_price'], row['open_date'], 
           row['close_date'], row['monthly_return'])
    mycursor.execute(sql, val)
    mydb.commit()